# 03-验证码
## 1.手机号是否重复 ajax
  *     1.1 接收参数 
  *     1.2 校验 判空 判正则
  *      1.3 数据库查询 User.objects.filter(mobile=mobile).count()
  *     1.4 返回前端Json
## 2.图片验证码
*     1.接收前端的 UUID  唯一标识符 校验正则
*     2.生成图片验证码---第三方包captach
*     3.保存redis数据库(1.配置redis库 2.django-redis 3步 导包 链接 setex)
*     4.返回给前端 图片的二进制流


## 3.短信验证码
*   1.接收 2个 mobile; 图片验证码img_code
*   2.验证码 img_code和redis存储的验证码 是否一致 (1.redis取出来(4步) 2.判断是否相等 3.redis img_code 删除) **千万注意 redis取出来类型 是 Bytes.decode()**
*   3.生成随机 6位 短信验证码内容 random.randit()
*   4.存储 随机6位 redis里面(3步 )
*   5.发短信---第三方容联云--
*   6.返回响应对象

#### 容联云
*   1.注册账号
*   2. 创建应用--appid
*   3. 开发文档 
    *   3.1  绑定测试账号--自己注册那个手机号
    *   3.2  Demo 粘贴过来改参数    
## 4.注册验证短信验证码
   *   1.接收参数 msg_code
   *   2.从redis取出 redis_code
   *   3. 对比
   *   4. jinja2 模板语言
## 5.后台避免频繁发送短信
## 6.redis管道操作








# 03-登录
## 1.异步发送短信
* 1. 消息队列 --- 生产者 ----消费者  pip install celery
* 2. 项目根目录--包celery_tasks ---main.py
* 3. main.py--->1. 导包 from celery import Celery
* 4.            2. app = Celery('celery_tasks')
* 5.            3. 消息对垒---celery_tasks--config.py--->broker_url="redis://127.0.0.1:6379/14"
* 6. main.py---> app.config_from_object('celery_tasks.config')
* 7. 生产者: ---celery_tasks-包sms--tasks.py ---- (@app.task)def send_sms_code_ccp(mobile,sms_code): 粘贴发短信的代码
* 8.main.py-->添加任务 自动查找--app.autodiscover_tasks(['celery_tasks.sms'])
* 9. verifications.views.---->send_sms_code_ccp.delay(mobile,sms_code)
* 10. 启动异步 任务,  在终端的 项目目录下---celery -A celery_tasks.main worker -l info
* 11. 报错---no module libs--tasks-9行--->main.py-- os.settings.Env()

## 2. 登录
* 1. 登录页面显示

* 1.后台接收 解析参数 :3个参数 form表单  非表单 ,headers
* 2. 校验 判空判正则
* 3. 判断用户名 和密码是否正确
* 4.保持登登录状态login()
* 5.跳转到首页

## 首页用户名显示
* 1. 从 登录页成功---传递参数--index,html
* 2. 在index.html --user_id->数据库 ---去取 
* 3. cookie-->登录-->index--cookie--取出来
* 4. request.user.username 即可

## 1.多账号登录,系统不满足条件--继承父类重写父类的认证方法
* 1. 多账号 User.objects.get(mobile=mobile) 但是: 由于以前使用的authenticate验证的; 所以 自定义认证
* 2. 百度, 看官方文档 自定义里面 认证系统 https://yiyibooks.cn/xx/Django_1.11.6/topics/auth/customizing.html
* 3. 1.自定义类 (重写authenticate函数) 2.settings.dev.py 修改认证后端

## 2.判断用户是否登录 
* 1. 自己写 if user.objects.get(username=username ) else:
* 2. django if user.is_authenticated else
* 3. login_required() 装饰器函数
* 4. LoginRequiredMixin; 配置重定向到 登录页的路由

### 3.实现流程
判断是否登录 is_authenticated  login_required()  
   1. 显示 用户中心页面 
   2. 继承扩展类 LoginRequiredMixin
   3. 404 /accounts/login/  dev.py 改配置 LOGIN_URL = '/login/'
   4.?next=/info/  views.py 登录功能 加了个判断
## 3. QQ 登录流程
*  1.用户点击 QQ按钮
*  2.后台获取QQ的登录网址
*  3. 如果QQ登录成功:
    *   1. 绑定过后台--->保持登录状态---设置cookie-- 重定向到首页
    *   2. 未绑定---->重定向绑定页面进行绑定
*  4.是否绑定的表--新建--- 1.qq给我的openid 2.user

### 2.QQ认证 oauth2.0协议
*   1. QQ登录网址 ?code
*   2. code---token
*   3. token--openid

### 3.QQ 登录--建关联表 openid --user
  *    1.utils.models.py  抽象基类
  *    2.创建oauth子应用
  *    3. oauth.models.py  定义关联模型类
  *    4.数据迁移
### 4.QQ 登录--获取qq登录网址
   *   1.导包 from QQLoginTool.QQtool import OAuthQQ   
   *   2.Oauth(appid,appkey,redirect_uri) --from django.conf import settings
   *   3.login_url = get_qq_url() 
   *   4. 前端: <@click="qq_login">
### 5..QQ 登录--code-->token-->openid
*   1. 实现回调的 网址 oauthback/?code=fadsfsdsdf
*   2. 解析参数code, request.GET.get('code')
*   3.code-->token
*   4.token-->openid 
#### 6.是否绑定openid
6. 判断是否绑定 openid
   绑定了 -- login(request,oathu_user.user)---response==redirecrt(recerse())--response.set_cookie
   未绑定--- 1.content={'openid':openid} 2. render

    
    









# 04-QQ登录
## 1.多账号登录,系统不满足条件--继承父类重写父类的认证方法
* 1. 多账号 User.objects.get(mobile=mobile) 但是: 由于以前使用的authenticate验证的; 所以 自定义认证
* 2. 百度, 看官方文档 自定义里面 认证系统 https://yiyibooks.cn/xx/Django_1.11.6/topics/auth/customizing.html
* 3. 1.自定义类 (重写authenticate函数) 2.settings.dev.py 修改认证后端

## 3. QQ 登录流程
*  1.用户点击 QQ按钮
*  2.后台获取QQ的登录网址
*  3. 如果QQ登录成功:
    *   1. 绑定过后台--->保持登录状态---设置cookie-- 重定向到首页
    *   2. 未绑定---->重定向绑定页面进行绑定
*  4.是否绑定的表 1.qq给我的openid 2.user

### 2.QQ认证 oauth2.0协议
*   1. QQ登录网址 ?code="ffdfgddgdg"
*   2. code---token=="232223223"
*   3. token--openid(就是QQ 给后端登录成功一个凭证)

### 3.QQ 登录--建关联表 openid --user
  *    1.utils.models.py  抽象基类
  *    2.创建oauth子应用
  *    3. oauth.models.py  定义关联模型类
  *    4.数据迁移
### 4.QQ 登录--获取qq登录网址
   *   1.导包 from QQLoginTool.QQtool import OAuthQQ   
   *   2.Oauth(appid,appkey,redirect_uri) --from django.conf import settings
   *   3.login_url = get_qq_url() 
   *   4. 前端: <@click="qq_login">
### 5..QQ 登录--code-->token-->openid
*   1. 实现回调的 网址 oauthback/?code=fadsfsdsdf
*   2. 解析参数code, request.GET.get('code')
*   3.code-->token
*   4.token-->openid 
#### 6.是否绑定openid
6. 判断是否绑定 openid
   绑定了 -- login(request,oathu_user.user)---response==redirecrt(recerse())--response.set_cookie
   未绑定--- 1.content={'openid':openid} 2. render
   
#### 7.绑定QQ登录用户
*   1.接收参数 request.POST

*   2.解析参数 校验 判空,正则

*   3.校验短信: 1.redis取出来 2.redis_sms_code.decode()

*   4. 判断用户是否存在 mobile=mobile
  如果存在 check_password()
  不存在 新建了;

*   5. openid和user绑定 ==新建一条记录

*   6.login() —set_cookie 重定向的首页

## 1.加解密工具类的封装
*   1. 加解密 openid
*    2.  utils.secret.py 对象方法
   
*   3. 未绑定的 给前端 openid 加密
*   4. 绑定用户的时候 , openid 从前端传入的时候 解密

## 2.展示 用户基本信息
  *    1. 新增email_active字段 ; ausers.models.py
  *    2. 数据迁移
  *    3. request.user.username
  *    4. 替换 user_center.info.html(1.替换昨天素材 2.替换今天素材报错 address; html搜索 修改 #)


## 3.添加邮箱
 *    request.user.email = email
 *    request.user.save()


## 4.异步发邮件
*     1. email包--tasks.py--- @app.task send_mail()
*     2. main.py -- 注册
*     3. 开启 celery服务
*     4. views.py -- send_verify_email.delay(email)
*     5. @app.task(bind=True,name="任务名字", retry_backoff=3) self.retry(exs=e,max_retries=5)
*     6.生成 激活的 链接
    *      1. tasks.py 新增参数 verify_url
    *      2. views.py 调用封装完毕函数 --generator_verify_url(request.user)
    *      3. ausers.utils.py --- generator_verify_url
    *     3.1 加密参数  3.2 拼接url


        
## 5.激活邮件:
  *      user.email_active =True
  *      user.save()



    


# 长期作业 :解析json数据<-->反向组合数据json
{
    "1":{
        "channels":[
            {"id":1, "name":"手机", "url":"http://shouji.jd.com/"},
            {"id":2, "name":"相机", "url":"http://www.itcast.cn/"}
        ],
        "sub_cats":[
            {
                "id":38, 
                "name":"手机通讯", 
                "sub_cats":[
                    {"id":115, "name":"手机"},
                    {"id":116, "name":"游戏手机"}
                ]
            },
            {
                "id":39, 
                "name":"手机配件", 
                "sub_cats":[
                    {"id":119, "name":"手机壳"},
                    {"id":120, "name":"贴膜"}
                ]
            }
        ]
    },
    "2":{
        "channels":[],
        "sub_cats":[]
    }
}


食堂吃饭在哪里--->钱 --->充卡--->打饭 刷卡
login_url--->code--->token--->openid




